# Two Data group example

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
from holoviews.operation.normalization import subcoordinate_group_ranges
from holoviews.operation.datashader import rasterize
from holoviews.plotting.links import RangeToolLink
from scipy.stats import zscore
import colorcet as cc

hv.extension('bokeh')

GROUP_EEG = 'EEG'
GROUP_POS = 'Position'
N_CHANNELS_EEG = 10
N_CHANNELS_POS = 3
N_SECONDS = 5
SAMPLING_RATE_EEG = 200
SAMPLING_RATE_POS = 25
INIT_FREQ = 2  # Initial frequency in Hz
FREQ_INC = 5  # Frequency increment
AMPLITUDE_EEG = 1000  # EEG amplitude multiplier
AMPLITUDE_POS = 2  # Position amplitude multiplier

# Generate time for EEG and position data
total_samples_eeg = N_SECONDS * SAMPLING_RATE_EEG
total_samples_pos = N_SECONDS * SAMPLING_RATE_POS
time_eeg = np.linspace(0, N_SECONDS, total_samples_eeg)
time_pos = np.linspace(0, N_SECONDS, total_samples_pos)

# Generate EEG timeseries data
def generate_eeg_data(index):
    return AMPLITUDE_EEG * np.sin(2 * np.pi * (INIT_FREQ + index * FREQ_INC) * time_eeg)

eeg_channels = [str(i) for i in np.arange(N_CHANNELS_EEG)]
eeg_data = np.array([generate_eeg_data(i) for i in np.arange(N_CHANNELS_EEG)])
eeg_df = pd.DataFrame(eeg_data.T, index=time_eeg, columns=eeg_channels)
eeg_df.index.name = 'Time'

# Generate position data
pos_channels = ['X', 'Y', 'Z'] # avoid lowercase 'x' and 'y' as channel/dimension names
pos_data = AMPLITUDE_POS * np.random.randn(N_CHANNELS_POS, total_samples_pos).cumsum(axis=1)
pos_df = pd.DataFrame(pos_data.T, index=time_pos, columns=pos_channels)
pos_df.index.name = 'Time'

## Create a Curve per data series

In [ ]:

def df_to_curves(df, kdim, vdim, color='black', group='EEG'):
    curves = []
    for i, (channel, channel_data) in enumerate(df.items()):
        ds = hv.Dataset((channel_data.index, channel_data), [kdim, vdim])
        curve = hv.Curve(ds, kdim, vdim, group=group, label=str(channel))
        curve.opts(
            subcoordinate_y=True, color=color if isinstance(color, str) else color[i], line_width=1, 
            hover_tooltips=hover_tooltips, tools=['xwheel_zoom'], line_alpha=.8,
        )
        curves.append(curve)
    return curves

hover_tooltips = [("Group", "$group"), ("Channel", "$label"), ("Time"), ("Value")]

vdim_EEG = hv.Dimension("Value", unit="µV")
vdim_POS = hv.Dimension("Value", unit="cm")
time_dim = hv.Dimension("Time", unit="s")

eeg_curves = df_to_curves(eeg_df, time_dim, vdim_EEG, color='black', group='EEG')
pos_curves = df_to_curves(pos_df, time_dim, vdim_POS, color=cc.glasbey_cool, group='POS')

# Combine EEG and POS curves into an Overlay
eeg_curves_overlay = hv.Overlay(eeg_curves, kdims="Channel")
pos_curves_overlay = hv.Overlay(pos_curves, kdims="Channel")
curves_overlay = (eeg_curves_overlay * pos_curves_overlay).opts(
    xlabel=time_dim.pprint_label, ylabel="Channel", show_legend=False, aspect=3, responsive=True,
)
curves_overlay

## Apply group-wise normalization

In [ ]:
normalized_overlay = subcoordinate_group_ranges(curves_overlay)
normalized_overlay

## Minimap

In [ ]:
y_positions = range(N_CHANNELS_EEG + N_CHANNELS_POS)

# Reindex the lower frequency DataFrame to match the higher frequency index
pos_df_interp = pos_df.reindex(eeg_df.index).interpolate(method='index')

# concatenate the EEG and interpolated POS data and z-score the full data array
z_data = zscore(np.concatenate((eeg_df.values, pos_df_interp.values), axis=1), axis=0).T

minimap = rasterize(hv.Image((time_eeg, y_positions , z_data), [time_dim, "Channel"], "Value"))
minimap = minimap.opts(
    cmap="RdBu_r", xlabel='', alpha=.7,
    yticks=[(y_positions[0], f'EEG {eeg_channels[0]}'), (y_positions[-1], f'POS {pos_channels[-1]}')],
    height=120, responsive=True, toolbar='disable', cnorm='eq_hist'
)
minimap

RangeToolLink(
    minimap, normalized_overlay, axes=["x", "y"],
    boundsx=(.5, 3), boundsy=(1.5, 12.5),
    intervalsx=(None, 3),
)

dashboard = (normalized_overlay + minimap).cols(1).opts(shared_axes=False)
dashboard

# NdOverlay and group, channel key to demo wide df issues

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import colorcet as cc
hv.extension('bokeh')

GROUP_EEG = 'EEG'
N_CHANNELS_EEG = 31
N_SECONDS = 5
SAMPLING_RATE_EEG = 100
INIT_FREQ = 2  # Initial frequency in Hz
FREQ_INC = 5  # Frequency increment
AMPLITUDE_EEG = 1000  # Amplitude multiplier

# Generate data
total_samples_eeg = N_SECONDS * SAMPLING_RATE_EEG
time_eeg = np.linspace(0, N_SECONDS, total_samples_eeg)
def generate_eeg_data(index):
    return AMPLITUDE_EEG * np.sin(2 * np.pi * (INIT_FREQ + index * FREQ_INC) * time_eeg)
eeg_channels = [str(i) for i in np.arange(N_CHANNELS_EEG)]
eeg_data = np.array([generate_eeg_data(i) for i in np.arange(N_CHANNELS_EEG)])
eeg_df = pd.DataFrame(eeg_data.T, index=time_eeg, columns=eeg_channels)
eeg_df.index.name = 'Time'

# Create plot
time_dim = hv.Dimension("Time", unit="s")
curves = {}
for col in eeg_df.columns:
    curve = hv.Curve(eeg_df[col], time_dim, hv.Dimension(col, label='Amplitude', unit='uV'),
                     label=str(col))
    curve = curve.opts(subcoordinate_y=True, tools=['xwheel_zoom', 'hover'], color='grey',
                      )
    curves['EEG', col] = curve

curves_overlay = hv.NdOverlay(curves, ["Group", "Channel"], sort=False).opts(
    xlabel=time_dim.pprint_label, ylabel="Channel", show_legend=False, aspect=3, responsive=True,
    min_height=600,
)
print(curves_overlay)
curves_overlay

# NdOverlay with wide df and hover_tooltips

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import colorcet as cc
hv.extension('bokeh')

GROUP_EEG = 'EEG'
N_CHANNELS_EEG = 31
N_SECONDS = 5
SAMPLING_RATE_EEG = 100
INIT_FREQ = 2  # Initial frequency in Hz
FREQ_INC = 5  # Frequency increment
AMPLITUDE_EEG = 1000  # Amplitude multiplier

# Generate data
total_samples_eeg = N_SECONDS * SAMPLING_RATE_EEG
time_eeg = np.linspace(0, N_SECONDS, total_samples_eeg)
def generate_eeg_data(index):
    return AMPLITUDE_EEG * np.sin(2 * np.pi * (INIT_FREQ + index * FREQ_INC) * time_eeg)
eeg_channels = [str(i) for i in np.arange(N_CHANNELS_EEG)]
eeg_data = np.array([generate_eeg_data(i) for i in np.arange(N_CHANNELS_EEG)])
eeg_df = pd.DataFrame(eeg_data.T, index=time_eeg, columns=eeg_channels)
eeg_df.index.name = 'Time'

# Create plot
time_dim = hv.Dimension("Time", unit="s")
curves = {}
for col in eeg_df.columns:
    curve = hv.Curve(eeg_df[col], time_dim, hv.Dimension(col, label='Amplitude', unit='uV'),
                     group='EEG', label=str(col))
    curve = curve.opts(subcoordinate_y=True, tools=['xwheel_zoom'], color='grey',
                      hover_tooltips = [("Group", "$group"), ("Channel", "$label"), ("Time"), ("Amplitude")])
    curves[('EEG', col)] = curve

curves_overlay = hv.NdOverlay(curves, ["Group", "Channel"], sort=False).opts(
    xlabel=time_dim.pprint_label, ylabel="Channel", show_legend=False, aspect=3, responsive=True,
    min_height=600, title='Multi-Chan TS'
)
print(curves_overlay)
curves_overlay